
# Backtest Strategi dengan Data Langsung dari OKX

Notebook ini memuat data harga langsung dari publik API OKX sehingga Anda tidak perlu lagi menyiapkan file CSV secara terpisah. Cukup atur parameter di sel konfigurasi dan jalankan seluruh notebook untuk menarik data, menjalankan strategi, dan melihat hasil backtest.



## Cara menggunakan
1. Jalankan sel impor di bawah ini.
2. Ubah parameter di bagian **Konfigurasi dataset & strategy** sesuai kebutuhan (instrumen, timeframe, dan rentang waktu).
3. Jalankan sel-sel berikutnya untuk menarik data dari OKX dan menjalankan strategi yang dipilih.


In [1]:

from __future__ import annotations

import json
import sys
import time
import socket
from pathlib import Path
from typing import Dict, List, Tuple
from urllib import error, parse, request

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython import get_ipython
from IPython.display import display

PROJECT_ROOT = Path('..').resolve()
for path in (PROJECT_ROOT, PROJECT_ROOT / 'src'):
    if str(path) not in sys.path:
        sys.path.append(str(path))

_ip = get_ipython()
if _ip is not None:
    try:
        _ip.run_line_magic('matplotlib', 'inline')
    except AttributeError:
        plt.switch_backend('Agg')
else:
    plt.switch_backend('Agg')

plt.style.use('seaborn-v0_8-darkgrid')

from src.strategy_backtest import SignalBacktester, get_strategy, list_strategies
from src.strategy_backtest.utils import sanitise_columns


In [2]:

# Konfigurasi dataset & strategy
ASSET_SYMBOL = 'ETHUSDT'
OKX_INSTRUMENT = 'ETH-USDT-SWAP'  # Format instId, contoh lain: BTC-USDT-SWAP, ETH-USDT
TIMEFRAME = '1H'  # Lihat dokumentasi OKX untuk opsi lain: 1m, 5m, 15m, 1H, 4H, 1D, dst.
START_TIME = '2025-01-01'  # Gunakan None untuk mengambil seluruh riwayat tersedia
END_TIME = '2025-11-14'    # Gunakan None untuk tanggal saat ini
REQUEST_LIMIT = 200        # Maksimum candle per permintaan (OKX mengizinkan hingga 300)
REQUEST_SLEEP = 0.2        # Jeda antar permintaan agar tidak terkena rate-limit

TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'

# Ganti nama strategy sesuai file di `src/strategy_backtest/strategies/`
STRATEGY_NAME = 'vwap'
# Opsional: override parameter default strategy
STRATEGY_PARAMS = {}



In [4]:
OKX_CANDLES_URL = 'https://www.okx.com/api/v5/market/candles'
OKX_REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (compatible; BacktestIndicator/1.0; +https://github.com/amri-qrypto/backtest-indicator)',
    'Accept': 'application/json',
}


def _to_millis(value):
    if value in (None, ''):
        return None
    ts = pd.Timestamp(value, tz='UTC')
    return int(ts.timestamp() * 1000)


def fetch_okx_ohlcv(
    inst_id: str,
    bar: str,
    start=None,
    end=None,
    limit: int = 200,
    pause: float = 0.2,
    request_timeout: float = 10.0,
) -> Tuple[pd.DataFrame, Dict[str, str]]:
    start_ms = _to_millis(start)
    end_ms = _to_millis(end)
    if end_ms is None:
        end_ms = int(pd.Timestamp.utcnow().timestamp() * 1000)

    params = {'instId': inst_id, 'bar': bar, 'limit': str(limit)}
    all_rows: List[List[str]] = []
    cursor = end_ms

    while True:
        query = params | {'before': str(cursor)}
        url = f"{OKX_CANDLES_URL}?{parse.urlencode(query)}"
        req = request.Request(url, headers=OKX_REQUEST_HEADERS)
        try:
            with request.urlopen(req, timeout=request_timeout) as resp:
                raw_body = resp.read().decode('utf-8')
        except (error.URLError, TimeoutError, socket.timeout) as exc:
            raise RuntimeError(f'Gagal terhubung ke OKX: {exc}') from exc

        try:
            payload = json.loads(raw_body)
        except json.JSONDecodeError as exc:
            raise RuntimeError('Respon OKX tidak valid') from exc

        if payload.get('code') != '0':
            message = f"Gagal mengambil data OKX: {payload.get('msg')} (code={payload.get('code')})"
            raise RuntimeError(message)

        rows = payload.get('data', [])
        if not rows:
            break

        all_rows.extend(rows)

        earliest = min(int(row[0]) for row in rows)
        reached_start = start_ms is not None and earliest <= start_ms
        exhausted = len(rows) < int(params['limit'])

        if reached_start:
            break
        if start_ms is None and exhausted:
            break

        next_cursor = earliest - 1
        if next_cursor <= 0:
            break
        cursor = next_cursor

        time.sleep(max(pause, 0))

    if not all_rows:
        raise RuntimeError('Tidak ada data OHLCV yang diterima dari OKX')

    df = pd.DataFrame(
        all_rows,
        columns=['ts', 'o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote', 'confirm'],
    )
    df = df.drop_duplicates(subset='ts')
    df['ts'] = pd.to_numeric(df['ts'], errors='coerce')
    df = df.dropna(subset=['ts'])
    df = df.sort_values('ts')

    if start_ms is not None:
        df = df[df['ts'] >= start_ms]
    if end_ms is not None:
        df = df[df['ts'] <= end_ms]

    numeric_cols = ['o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote']
    for column in numeric_cols:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    df['confirm'] = df['confirm'].astype(int)

    df['time'] = pd.to_datetime(df['ts'], unit='ms', utc=True).tz_convert(None)
    df = df.drop(columns=['ts'])
    df = df.dropna(subset=['time'])

    rename_map = {
        'o': 'open',
        'h': 'high',
        'l': 'low',
        'c': 'close',
        'vol': 'volume',
        'volCcy': 'volume_ccy',
        'volCcyQuote': 'volume_quote',
        'confirm': 'confirm',
    }
    df = df.rename(columns=rename_map)
    raw_mapping: Dict[str, str] = {'time': 'ts'} | {new: original for original, new in rename_map.items()}
    sanitised_cols, _ = sanitise_columns(df.columns)
    column_mapping = {alias: raw_mapping[col_name] for alias, col_name in zip(sanitised_cols, df.columns)}
    df.columns = sanitised_cols
    df = df.set_index('time').sort_index()

    if start_ms is not None and not df.empty:
        requested_start = pd.Timestamp(start_ms, unit='ms', tz='UTC')
        actual_start = pd.Timestamp(df.index.min(), tz='UTC')
        if actual_start > requested_start:
            print(f"⚠️ Data OKX tersedia mulai {actual_start.tz_convert(None)} (lebih lambat dari START_TIME).")

    return df, column_mapping


data, column_mapping = fetch_okx_ohlcv(
    inst_id=OKX_INSTRUMENT,
    bar=TIMEFRAME,
    start=START_TIME,
    end=END_TIME,
    limit=REQUEST_LIMIT,
    pause=REQUEST_SLEEP,
)

print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
if not data.empty:
    print(f"Rentang waktu: {data.index.min()} -> {data.index.max()}")
print('Contoh mapping kolom (sanitised -> OKX):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

display(data.head())


RuntimeError: Gagal terhubung ke OKX: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:

available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_PARAMS)
print('Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print('Parameter default:')
for key, value in strategy.metadata.parameters.items():
    print(f'  {key}: {value}')
if not strategy.metadata.parameters:
    print('  (tidak ada parameter default eksplisit)')
print('Parameter aktif:')
for key, value in strategy.params.items():
    print(f'  {key}: {value}')
if not strategy.params:
    print('  (menggunakan nilai default)')


In [ ]:

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
display(signals.head())

backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

print('Metrik performa:')
for key, value in outputs.metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('Ringkasan trade:')
for key, value in outputs.trade_summary.items():
    print(f'- {key}: {value}')

display(outputs.trades.head())


In [ ]:

def _as_bool(series: pd.Series | None) -> pd.Series:
    if series is None:
        return pd.Series(False, index=data.index)
    aligned = series.reindex(data.index)
    return aligned.fillna(False).astype(bool)

close_prices = data[PRICE_COLUMN]
long_entries = _as_bool(signals.get('long_entry'))
long_exits = _as_bool(signals.get('long_exit'))
short_entries = _as_bool(signals.get('short_entry'))
short_exits = _as_bool(signals.get('short_exit'))

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(close_prices.index, close_prices, label='Close', color='black', linewidth=1.2)

if long_entries.any():
    ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry', zorder=5)
if short_entries.any():
    ax.scatter(close_prices.index[short_entries], close_prices[short_entries], marker='v', color='red', label='Short Entry', zorder=5)
if long_exits.any():
    ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='tab:blue', label='Long Exit', zorder=6)
if short_exits.any():
    ax.scatter(close_prices.index[short_exits], close_prices[short_exits], marker='^', color='tab:orange', label='Short Exit', zorder=6)

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend(loc='upper left', ncol=2)
fig.tight_layout()
display(fig)
plt.close(fig)

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)


In [ ]:
display(outputs.results.head())
